# Notebook de Exploração (word2vec)

Nesse arquivo, realizamos uma análise detalhada e experimentação com diversos modelos de aprendizado de máquina para determinar qual oferece o melhor desempenho para a nossa tarefa específica. Este processo inclui a preparação e a limpeza dos dados, o balanceamento das classes, a extração de características relevantes e a avaliação de múltiplos algoritmos. Diversos testes e validações são conduzidos para refinar os modelos e otimizar seus parâmetros. O objetivo final é identificar o modelo mais eficaz e robusto para ser utilizado em produção, garantindo uma análise precisa e confiável dos dados.

### Justificativa para a Escolha do Modelo Word2Vec

Para este projeto, optamos por utilizar o modelo Word2Vec pré-treinado do Google. A principal razão é que nossa base de dados da Uber está em inglês, e o modelo do Google foi treinado em um grande corpus de texto em inglês, garantindo uma melhor representação semântica das palavras em nosso contexto. Embora existam outras alternativas, como o Word2Vec do NILC treinado em português, a escolha do modelo do Google é mais adequada para nossa necessidade específica, pois oferece uma vasta quantidade de dados em inglês, apesar de ter menos alternativas de modelos com dimensões diferentes.

## Baixar Pandas para importar CSV

In [ ]:
import pandas as pd

Importando os dados

In [ ]:
df = pd.read_csv('../Notebooks/dados.csv')
df.head()

# Análise Exploratória do Corpus

**Introdução**

A análise exploratória de dados é um passo fundamental para desenvolver modelos que classificam sentimentos, especialmente quando lidamos com dados complexos como comentários de usuários. Neste projeto, o grupo Moodfy estudou um conjunto de comentários sobre a Uber retirados da plataforma X (antes conhecida como Twitter). O objetivo dessa análise inicial era entender melhor os dados, descobrir padrões e preparar a base para criar um modelo que possa classificar os comentários como positivos, negativos ou neutros. Esse trabalho inicial é crucial para garantir que o modelo final seja preciso e eficaz, influenciando decisões importantes sobre como atender e se comunicar com os clientes.


**Explicação dos Passos da Análise Exploratória Feita pelo Grupo Moodfy**

1. Distribuição de Sentimentos: O grupo Moodfy começou analisando como os sentimentos estavam distribuídos entre os comentários para ver se havia um equilíbrio entre positivos, negativos e neutros. Essa checagem é importante porque um desequilíbrio pode fazer o modelo futuro pender para o lado mais comum.

2. Análise de Palavras Comuns em Comentários: Eles identificaram quais palavras apareciam mais em cada tipo de sentimento. Esse passo ajuda a entender quais temas são frequentes e como certas palavras podem influenciar a classificação dos sentimentos.

3. Análise de Comprimento dos Comentários: O grupo investigou se o tamanho dos comentários estava relacionado com os sentimentos expressados. Comentários mais longos podem indicar sentimentos mais fortes.

4. Frequência de Palavras por Sentimento: Essa etapa detalhou mais a análise anterior, quantificando quantas vezes certas palavras apareciam nos diferentes sentimentos. Isso foi útil para ver se algumas palavras muito comuns, que não adicionam muito significado, estavam influenciando os resultados.

5. Correlação entre Comprimento do Comentário e Sentimento: Eles também verificaram se havia uma relação entre o tamanho dos comentários e o tipo de sentimento, para entender se pessoas mais satisfeitas ou insatisfeitas tendem a escrever mais.

6. Média de Comprimento dos Comentários por Sentimento: Por último, o grupo calculou o comprimento médio dos comentários para cada sentimento, buscando identificar comentários muito longos ou curtos que fogem do comum, o que pode sugerir a necessidade de ajustar os dados antes de criar o modelo.

#### Importação de bibliotecas de análise gráfica

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter

#### Análise Exploratória do Corpus

Criação de Dataframe exclusivo para análise exploratória do corpus

In [ ]:
df_analysis = pd.read_csv('../Notebooks/dados.csv')

df_analysis.head()

**1. Distribuição de Sentimentos**


Este gráfico de barras mostra a frequência de cada categoria de sentimento nos dados. Os sentimentos estão categorizados como -1 (negativo), 0 (neutro) e 1 (positivo). O gráfico ajuda a visualizar rapidamente a proporção de comentários em cada categoria, permitindo uma análise quantitativa rápida da natureza geral dos comentários no dataset. A visualização utiliza cores diferentes para cada categoria para facilitar a distinção: vermelho para negativo, cinza para neutro e azul para positivo. Esta análise é baseada no autoestudo "Como Fazer Análise de Sentimentos com Dados Textuais", onde aprendemos a aplicar métodos estatísticos para entender a distribuição de sentimentos em dados textuais.

In [ ]:
def plot_sentiment_distribution(df_analysis):
    """
    Cria um gráfico de barras para mostrar a distribuição dos sentimentos nos comentários.
    
    Args:
        df_analysis (DataFrame): DataFrame contendo os dados dos sentimentos.
    
    Returns:
        None: Exibe o gráfico de barras com a distribuição dos sentimentos.
    """
    # Calcula a contagem de cada sentimento
    sentiment_counts = df_analysis['sentiment'].value_counts().sort_index()

    # Gráfico de barras
    plt.bar(sentiment_counts.index, sentiment_counts.values, color=['red', 'grey', 'blue'])
    plt.xlabel('Sentimento')
    plt.ylabel('Frequência de comentários')
    plt.title('Distribuição de Sentimentos')
    plt.xticks([-1, 0, 1], ['Negativo', 'Neutro', 'Positivo'])
    plt.show()

plot_sentiment_distribution(df_analysis)

**2. Análise de Palavras Comuns em Comentários**


Utilizando a técnica de nuvem de palavras, este gráfico destaca as palavras mais comuns em comentários de diferentes sentimentos.


In [ ]:
def generate_wordcloud(data, title):
    """Gera uma nuvem de palavras para visualizar as palavras mais comuns em comentários.

    Args:
        data (Series): Série de Pandas contendo os comentários.
        title (str): Título para a nuvem de palavras.

    Returns:
        None: Exibe a nuvem de palavras.
    """
    wc = WordCloud(background_color='white', max_words=200)
    plt.figure(figsize=(10, 6))
    plt.imshow(wc.generate(' '.join(data)), interpolation='bilinear')
    plt.title(title)
    plt.axis('off')
    plt.show()

# Gerar Word Clouds para cada sentimento
generate_wordcloud(df_analysis[df_analysis['sentiment'] == -1]['comment'], 'Palavras mais comuns em comentários negativos')
generate_wordcloud(df_analysis[df_analysis['sentiment'] == 0]['comment'], 'Palavras mais comuns em comentários neutros')
generate_wordcloud(df_analysis[df_analysis['sentiment'] == 1]['comment'], 'Palavras mais comuns em comentários positivos')

**3. Análise de Comprimento dos Comentários**

Este gráfico de caixa (boxplot) ilustra a distribuição do comprimento dos comentários com base nos sentimentos expressos, categorizados em negativo (-1), neutro (0) e positivo (1). Cada caixa no gráfico representa a distribuição dos comprimentos de comentários para uma categoria de sentimento específica, oferecendo uma visão sobre a mediana, os quartis e os valores extremos (outliers) de comprimento para cada grupo.


In [ ]:
def plot_comment_length_by_sentiment(df_analysis):
    """Gera um gráfico de caixa para visualizar a distribuição do comprimento dos comentários por sentimento.

    Args:
        df (DataFrame): DataFrame contendo as colunas 'comment' e 'sentiment', onde 'comment' são os comentários
                        e 'sentiment' são os sentimentos associados aos comentários.

    Returns:
        None: Exibe o gráfico de caixa mostrando o comprimento dos comentários distribuídos por sentimentos.
    """
    # Calcula o comprimento de cada comentário
    df_analysis['comment_length'] = df_analysis['comment'].apply(len)

    # Configura e exibe o gráfico de caixa
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='sentiment', y='comment_length', data=df_analysis)
    plt.title('Comprimento dos Comentários por Sentimento')
    plt.xlabel('Sentimento')
    plt.ylabel('Comprimento do Comentário (N° de caraceteres)')
    plt.xticks(ticks=[0, 1, 2], labels=['Negativo (-1)', 'Neutro (0)', 'Positivo (1)'])
    plt.show()

# chamada da função:
plot_comment_length_by_sentiment(df_analysis)

**4. Frequência de Palavras por Sentimento**


Este gráfico de barras compara a frequência das palavras mais comuns em comentários negativos e neutros. As barras indicam quantas vezes cada palavra foi mencionada, proporcionando uma visualização clara das diferenças no vocabulário usado em diferentes estados emocionais. Analisar essas frequências ajuda a entender os temas predominantes e possíveis áreas de insatisfação ou discussões gerais que não envolvem sentimentos fortes.


In [ ]:
def analyze_word_frequency_pipeline(df):
    """Analisa e visualiza as palavras mais comuns em comentários categorizados por sentimentos negativos, neutros e positivos.

    Args:
        df_analysis (DataFrame): DataFrame contendo as colunas 'comment' e 'sentiment', onde 'comment' são os comentários
                        e 'sentiment' identifica o sentimento do comentário.

    Returns:
        None: Gera gráficos de barras mostrando as palavras mais comuns para cada categoria de sentimento.
    """
    # Função para contar palavras em um texto
    def count_words(text):
        words = text.split()
        return Counter(words)

    # Aplicação da função de contagem de palavras e agregação por sentimento
    df['words'] = df['comment'].apply(count_words)
    negative_words = sum(df[df['sentiment'] == -1]['words'], Counter())
    neutral_words = sum(df[df['sentiment'] == 0]['words'], Counter())
    positive_words = sum(df[df['sentiment'] == 1]['words'], Counter())

    # Seleção das 10 palavras mais comuns em cada categoria de sentimento
    most_common_neg = negative_words.most_common(10)
    most_common_neu = neutral_words.most_common(10)
    most_common_pos = positive_words.most_common(10)

    # Criação de gráficos de barras para cada categoria de sentimento
    fig, ax = plt.subplots(3, 1, figsize=(10, 8))

    ax[0].bar([word[0] for word in most_common_neg], [word[1] for word in most_common_neg], color='red')
    ax[0].set_title('Palavras mais comuns em comentários negativos')
    ax[0].set_ylabel('Frequência De Repetição da Palavra')

    ax[1].bar([word[0] for word in most_common_neu], [word[1] for word in most_common_neu], color='grey')
    ax[1].set_title('Palavras mais comuns em comentários neutros')
    ax[1].set_ylabel('Frequência De Repetição da Palavra')

    ax[2].bar([word[0] for word in most_common_pos], [word[1] for word in most_common_pos], color='green')
    ax[2].set_title('Palavras mais comuns em comentários positivos')
    ax[2].set_ylabel('Frequência De Repetição da Palavra')

    plt.tight_layout()
    plt.show()

# chamada da função:
analyze_word_frequency_pipeline(df_analysis)


**5. Correlação entre Comprimento do Comentário e Sentimento**


Este gráfico de dispersão explora se o comprimento dos comentários varia com o sentimento expresso. Cada ponto representa um comentário, posicionado de acordo com seu sentimento (negativo, neutro) e comprimento. A visualização ajuda a identificar se comentários mais longos tendem a ser negativos, positivos, ou neutros, fornecendo insights sobre como os usuários se expressam em diferentes contextos emocionais.

In [ ]:
def plot_comment_length_sentiment_correlation(df_analysis):
    """Gera um gráfico de dispersão para examinar a correlação entre o comprimento dos comentários e os sentimentos expressos.

    Args:
        df_analysis (DataFrame): DataFrame contendo as colunas 'sentiment' e 'comment_length', onde 'sentiment' indica o sentimento
                        do comentário e 'comment_length' é o comprimento do comentário medido em número de caracteres.

    Returns:
        None: Exibe o gráfico de dispersão.
    """
    plt.figure(figsize=(8, 5))
    plt.scatter(df_analysis['sentiment'], df_analysis['comment_length'], alpha=0.5)
    plt.title('Correlação entre Comprimento do Comentário e Sentimento')
    plt.xlabel('Tipo de Sentimento')
    plt.ylabel('Comprimento do Comentário - (N° Caracteres)')
    plt.xticks([-1, 0, 1], ['Negativo', 'Neutro', 'Positivo'])
    plt.grid(True)
    plt.show()

# chamada da função:
plot_comment_length_sentiment_correlation(df_analysis)


**6. Média de Comprimento dos Comentários por Sentimento**

Esse é um gráfico de barras que mostra a média de comprimento dos comentários para cada categoria de sentimento. Este gráfico pode indicar se sentimentos mais fortes (positivos ou negativos) levam a comentários mais detalhados.

In [ ]:
def plot_average_comment_length_by_sentiment(df_analysis):
    """Calcula e visualiza a média de comprimento dos comentários por sentimento em um gráfico de barras.

    Args:
        df_analysis (DataFrame): DataFrame contendo as colunas 'sentiment' e 'comment_length', onde 'sentiment' indica o sentimento
                        do comentário e 'comment_length' é o comprimento do comentário.

    Returns:
        None: Exibe o gráfico de barras mostrando a média de comprimento dos comentários para cada sentimento.
    """
    # Cálculo da média de comprimento dos comentários por sentimento
    average_lengths = df_analysis.groupby('sentiment')['comment_length'].mean()

    # Configuração e exibição do gráfico de barras
    plt.figure(figsize=(10, 6))
    sns.barplot(x=average_lengths.index, y=average_lengths.values, palette='coolwarm')
    plt.title('Média de Comprimento dos Comentários por Sentimento')
    plt.xlabel('Sentimento')
    plt.ylabel('Média de Comprimento do Comentário (N° de Caracteres)')
    plt.xticks(ticks=[0, 1, 2], labels=['Negativo (-1)', 'Neutro (0)', 'Positivo (1)'])
    plt.show()

# chamada da função:
plot_average_comment_length_by_sentiment(df_analysis)


In [ ]:
df_analysis.head(10)

# Pré-Processamento

O pré-processamento é uma etapa fundamental em um modelo de machine learning, responsável por preparar e organizar os dados antes de serem alimentados ao algoritmo de aprendizado. Ele inclui uma série de técnicas e procedimentos, como normalização, padronização, tratamento de dados faltantes e seleção de características relevantes. A funcionalidade do pré-processamento é melhorar a qualidade dos dados, tornando-os mais adequados e representativos para o modelo de machine learning. Isso ajuda a evitar problemas como overfitting, garantir que o modelo possa generalizar bem para dados novos e melhorar a eficácia do aprendizado.

### Importação de bibliotecas

In [ ]:
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk import pos_tag
import string
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.preprocessing import FunctionTransformer
from spellchecker import SpellChecker
from sklearn.metrics import accuracy_score, classification_report
from gensim.models import KeyedVectors, FastText, Word2Vec
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cosine

# Baixar os recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

### Tokenização

A tokenização é o processo de dividir um texto em unidades menores chamadas tokens. Esses tokens podem ser palavras individuais, partes de palavras ou até mesmo caracteres, dependendo do nível de granularidade desejado. A tokenização é uma etapa fundamental no processamento de linguagem natural (PLN), sendo essencial para a preparação, análise e manipulação de texto em uma variedade de aplicações, incluindo análise de sentimento, classificação de texto e tradução automática. Ao dividir o texto em tokens, os dados tornam-se estruturados e adequados para análise, facilitando a extração de informações e a modelagem de soluções de PLN.

In [ ]:
def tokenize_text(text):
    """
    Tokeniza o DataFrame em uma lista de tokens.

    Args:
    text (DataFrame): Comentários a serem tokenizados.

    Returns:
    list: Lista de tokens.
    """
    tokens = word_tokenize(text.lower())
    return tokens

### Lemmatização

A lemmatização é o processo de reduzir palavras a sua forma base ou lema, considerando o contexto e a morfologia da língua. Essa técnica é importante em PLN para tratar diferentes formas de uma palavra como iguais, como "corre" e "correu" ambas reduzidas a "correr". Isso simplifica o processamento de texto e melhora a análise em tarefas como recuperação de informações e análise de sentimento.

In [ ]:
def get_wordnet_pos(treebank_tag):
    """
    Retorna o tag correspondente do WordNet para o tag do Treebank do Penn.
    """
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN 

def lemmatize_tokens_with_pos(tokens):
    """
    Lemmatiza uma lista de tokens baseando-se em sua parte do discurso.

    Args:
    tokens (list of str): Tokens a serem lematizados.

    Returns:
    list: Lista de lemas das palavras.
    """
    lemmatizer = WordNetLemmatizer()

    # Tokeniza e aplica POS tagging
    tagged_tokens = pos_tag(tokens)

    # Lemmatiza usando a parte do discurso
    lemmas = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in tagged_tokens]
    return lemmas

### Retirar pontuações

A remoção de pontuação é um processo utilizado no pré-processamento de texto que visa eliminar caracteres de pontuação, como vírgulas, pontos e pontos de exclamação, de um texto. Isso é feito para limpar o texto e reduzir a dimensionalidade dos dados, facilitando a análise e a modelagem. Ao remover a pontuação, os tokens resultantes contêm apenas palavras ou partes de palavras, tornando-os mais adequados para tarefas de processamento de linguagem natural.

In [ ]:
def remove_punctuation_from_tokens(tokens):
    """
    Remove pontuações de uma lista de tokens e exclui os tokens que consistem exclusivamente de caracteres de pontuação.

    Args:
    tokens (list): Lista de tokens a serem processados.

    Returns:
    list: Lista de tokens sem pontuações.
    """
    # Regex para identificar pontuações
    regex_punctuation = re.compile('[%s]' % re.escape(string.punctuation))

    # Remove pontuações de cada token e filtra tokens que ficaram vazios ou são apenas pontuações
    tokens_no_punct = [regex_punctuation.sub('', token) for token in tokens]
    tokens_no_punct = [token for token in tokens_no_punct if token.strip() != '']

    return tokens_no_punct

# Exemplo de uso da função:
tokens = ["hello!", "world...", "#amazing", "test,", ":)", "a", "."]
filtered_tokens = remove_punctuation_from_tokens(tokens)
print(filtered_tokens)


### Retirar números

A remoção de números é um passo comum no pré-processamento de texto que consiste em eliminar todos os caracteres numéricos de um texto. Isso é feito para limpar o texto de informações numéricas que podem não ser relevantes para a análise ou para garantir que as palavras sejam tratadas de maneira uniforme durante a tokenização. Ao remover números, os tokens resultantes contêm apenas palavras e outros caracteres não numéricos, simplificando o texto para análise e modelagem.

In [ ]:
def remove_numbers_from_tokens(tokens):
    """
    Remove todos os dígitos de uma lista de tokens e remove os tokens que consistem exclusivamente de números.

    Args:
    tokens (list): Lista de tokens a serem processados.

    Returns:
    list: Lista de tokens sem números.
    """
    # Remover dígitos de cada token e depois filtrar os tokens que são apenas números ou ficaram vazios
    tokens_no_numbers = [re.sub(r'\d+', '', token) for token in tokens]
    tokens_no_numbers = [token for token in tokens_no_numbers if token.strip() != '']
    return tokens_no_numbers

# Exemplo de uso da função:
tokens = ["hello123", "world", "2023", "test", "12345"]
filtered_tokens = remove_numbers_from_tokens(tokens)
print(filtered_tokens)


### Remoção de Stop Words

Stopwords são palavras que são frequentemente removidas durante o pré-processamento de texto em tarefas de Processamento de Linguagem Natural (PLN). Essas palavras são geralmente as mais comuns em um idioma, como ‘é’, ‘em’, ‘um’, ‘e’ em português, ou ‘is’, ‘in’, ‘a’, ‘and’ em inglês, e tendem a aparecer em quase todos os documentos de um corpus.

A remoção de stopwords é uma prática comum porque essas palavras, embora muito frequentes, geralmente não carregam muito significado e podem adicionar ruído aos dados. Além disso, removendo-as, podemos reduzir o tamanho do nosso vocabulário e, consequentemente, o espaço de recursos, tornando nossos modelos de PLN mais eficientes.

In [ ]:
class StopwordRemover(BaseEstimator, TransformerMixin):
    """
    Uma classe transformadora para remover stopwords e aplicar transformações de texto específicas,
    incluindo a remoção de aspas simples e duplas, e a transformação de 's em is.
    
    Args:
    language (str): Idioma das stopwords a serem removidas.
    
    Returns:
    list: Lista de comentários com stopwords removidas e transformações de texto aplicadas.
    """
    def __init__(self, language='english'):
        self.stopwords = set(stopwords.words(language))
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        processed = []
        for comment in X:
            # Remover aspas simples e duplas
            comment = re.sub(r"\'", "", comment)  # Remove todas as aspas simples
            comment = re.sub(r"\"", "", comment)  # Remove todas as aspas duplas

            # Substituir 's por is (considerando casos como it's, he's etc.)
            comment = re.sub(r"\'s\b", "is", comment)  # Substitui 's por is
            
            # Tokenização e remoção de stopwords
            tokens = word_tokenize(comment)
            filtered_tokens = [word for word in tokens if word.lower() not in self.stopwords]
            
            # Juntar os tokens filtrados de volta em uma string
            processed_comment = ' '.join(filtered_tokens)
            processed.append(processed_comment)
        
        return processed

# Exemplo de uso da classe:
X = ["It's a good day!", "He didn't like the movie's plot.", "\"That's great!\", he said."]
stopword_remover = StopwordRemover()
X_transformed = stopword_remover.transform(X)
print(X_transformed)


### Remoção de links


A remoção de links é uma etapa de pré-processamento em tarefas de Processamento de Linguagem Natural (PLN) que envolve a eliminação de URLs ou links de um texto. Isso é feito para reduzir o ruído nos dados e focar nas palavras e frases que carregam mais significado.

Links geralmente não contribuem para a semântica de um texto e podem ser bastante variados e únicos, o que pode adicionar ruído e aumentar a dimensionalidade dos dados. Além disso, os links podem levar a conteúdo externo que está fora do contexto do texto atual, tornando a análise mais complexa.

In [ ]:
# Definir a função para remover URLs
def remove_urls(text):
    """
    Remove URLs de um texto fornecido.

    Args:
    texto (list): O texto de entrada contendo URLs.

    Returns:
    list: Lista de tokens com URLs removidas.
    """
    return re.sub(r'http\S+|www.\S+', '', text, flags=re.MULTILINE)

### Balanceamento das Classes de Dados

O balanceamento das classes de dados é uma prática crucial em aprendizado de máquina para garantir que cada categoria seja representada de forma equitativa. Isso previne que o modelo desenvolva viés em direção à classe majoritária e melhora a precisão geral em prever categorias menos representadas. 

Na base de dados em questão, realizamos um balanceamento focado nas dinâmicas de desproporção entre as classes de sentimentos negativos e positivos. Primeiro, removemos os outliers baseando-nos apenas na distribuição dos próprios dados negativos para eliminar comentários atípicos que poderiam distorcer a análise. Após essa filtragem, procedemos com uma redução estratégica, eliminando aleatoriamente 40% dos dados negativos restantes. Além disso, para tratar a sub-representação dos sentimentos positivos, duplicamos os dados dessa classe. Essa abordagem combinada não só equilibra a presença das classes no dataset mas também facilita o treinamento de modelos mais justos e eficazes, evitando o viés em direção a qualquer classe específica.

In [ ]:
def remove_outliers_and_sample_negatives_and_duplicate_positives(df):
    """
    Remove outliers do comprimento dos comentários para a classe de sentimentos negativos,
    baseando-se nos quartis da própria classe negativa, e remove aleatoriamente 40% dos dados negativos
    restantes para ajudar no balanceamento das classes. Adicionalmente, duplica os dados da classe positiva
    para melhor balanceamento.

    Args:
    df (DataFrame): DataFrame contendo as colunas 'sentiment', 'comment', e 'comment_length'.

    Returns:
    DataFrame: DataFrame com outliers removidos da classe negativa, redução estratégica de 40% dos negativos,
    e duplicação dos dados positivos.
    """
    # Adiciona a coluna 'comment_length' ao DataFrame se não existir
    if 'comment_length' not in df.columns:
        df['comment_length'] = df['comment'].apply(len)
    
    # Filtra os dados para a classe negativa
    negativos = df[df['sentiment'] == -1]
    
    # Calcula os quartis apenas para a classe negativa
    Q1 = negativos['comment_length'].quantile(0.25)
    Q3 = negativos['comment_length'].quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR

    # Filtra os outliers na classe negativa baseado no limite calculado
    negativos_filtrados = negativos[negativos['comment_length'] <= upper_bound]
    
    # Amostragem aleatória para remover 40% dos dados negativos filtrados
    negativos_reduzidos = negativos_filtrados.sample(frac=0.6, random_state=42)

    # Duplica os dados da classe positiva
    positivos = df[df['sentiment'] == 1]
    positivos_duplicados = pd.concat([positivos] * 2, ignore_index=True)
    
    # Combina os dados reduzidos de sentimentos negativos com as outras classes
    df_final = pd.concat([negativos_reduzidos, df[df['sentiment'] == 0], positivos_duplicados], ignore_index=True)
    
    return df_final


### Pipeline


Pipeline é um conceito essencial em diversos campos, incluindo tecnologia da informação, manufatura, logística e até mesmo em processos criativos. Em termos gerais, refere-se a uma sequência de etapas interconectadas e ordenadas, onde o resultado de cada etapa serve como entrada para a próxima. Essas etapas podem ser atividades, operações ou processos específicos. O objetivo principal de um pipeline é otimizar a eficiência e a produtividade, dividindo um trabalho complexo em partes menores e mais gerenciáveis, permitindo que cada etapa seja executada de forma independente e simultânea. Isso não apenas acelera o processo, mas também permite melhorias contínuas em cada etapa individualmente, resultando em um produto final de maior qualidade.

In [ ]:
# Definir o pipeline para pré-processamento dos comentários
pipeline = Pipeline([
    ('url_remover', FunctionTransformer(lambda x: x.apply(remove_urls))),
    ('tokenizer', FunctionTransformer(lambda x: x.apply(tokenize_text))),
    ('lemmatizacao', FunctionTransformer(lambda x: x.apply(lemmatize_tokens_with_pos))),
    ('punctuation_remover', FunctionTransformer(lambda x: x.apply(remove_punctuation_from_tokens))),
    ('number_remover', FunctionTransformer(lambda x: x.apply(remove_numbers_from_tokens))),
    ('stopwords_remover', FunctionTransformer(lambda x: x.apply(StopwordRemover().transform))),
    ('join_tokens', FunctionTransformer(lambda x: x.apply(remove_stopwords))),
])

# Primeiro aplicar a remoção de outliers e a redução de dados negativos
df = remove_outliers_and_sample_negatives_and_duplicate_positives(df)

# Aplicar o pipeline ao DataFrame existente
df['comment'] = pipeline.fit_transform(df['comment'])
print(df)


### Exportar CSV

In [ ]:
df.to_csv('pipeline_comments.csv', index=False)

# Análise dos Corpus Após o Pré-Processamento

In [ ]:
plot_sentiment_distribution(df)

In [ ]:
generate_wordcloud(df[df['sentiment'] == -1]['comment'], 'Palavras mais comuns em comentários negativos')

In [ ]:
plot_comment_length_by_sentiment(df)

In [ ]:
analyze_word_frequency_pipeline(df)

In [ ]:
plot_comment_length_sentiment_correlation(df)

In [ ]:
plot_average_comment_length_by_sentiment(df)

# Word2Vec Skip-gram

### Vetores pré-treinados

Uso de vetores pré-treinados como ponto de partida para o treinamento do modelo Skip-gram

In [ ]:
skipgram_pretrained = "../src/GoogleNews-vectors-negative300.bin"

# Carregar modelo pré-treinado
modelo_skipgram = KeyedVectors.load_word2vec_format(skipgram_pretrained, binary=True)

### Teste com vetores pré-treinados

Testando a similaridade entre duas palavras usando o modelo pré-treinado para avaliar sua utilidade antes de proceder com o treinamento do nosso próprio modelo.

In [ ]:
wordvec_word1 = modelo_skipgram['dog']
wordvec_word2 = modelo_skipgram['puppy']

# Calculando a similaridade
similarity = 1 - cosine(wordvec_word1, wordvec_word2)
print(f"A similaridade entre 'dog' e 'puppy' é: {similarity}")

### Treinamento do modelo Skip-gram com nosso corpus

In [ ]:
corpus = df['comment'].apply(lambda x: word_tokenize(x.lower()) if isinstance(x, str) else [])

sentences = corpus.tolist()

# Treinar o modelo Word2Vec Skip-gram
model_sg = Word2Vec(sentences, vector_size=100, window=5, min_count=1, sg=1, workers=4, epochs=10)

# Salvar o modelo treinado
model_sg.wv.save_word2vec_format('word2vec_skipgram_model.bin', binary=True)

### Calculando e exibindo similaridade com o modelo treinado

Verificando a similaridade entre duas palavras para entender como o modelo está performando com os dados do projeto

In [ ]:
palavra1 = 'like'
palavra2 = 'hate'

similaridade_sg = model_sg.wv.similarity(palavra1, palavra2)
print(f'A similaridade entre "{palavra1}" e "{palavra2}" é: {similaridade_sg}')

### Função para somar os vetores de palavras da base

Como nossa base é pautada em frases e não em palavras soltas, para que o modelo word2ved faça mais sentido, nós devemos fazer uma função para somar as palavras presentes em determinada frase e assim conseguir um vetor resultade da frase.

In [ ]:
# Carregar o modelo treinado
model_sg = KeyedVectors.load_word2vec_format('word2vec_skipgram_model.bin', binary=True, unicode_errors='ignore')

# Função para sumarizar vetores de palavras
def sum_word_vectors_sg(sentence):
    sentence_vector = np.zeros(model_sg.vector_size)
    words_count = 0

    for word in sentence:
        if word in model_sg:
            sentence_vector += model_sg[word]
            words_count += 1

    if words_count != 0:
        sentence_vector /= words_count

    return sentence_vector

# Calcular e imprimir os vetores das sentenças
sentences_vectors_sg = [sum_word_vectors_sg(sentence) for sentence in sentences]
for i, vector in enumerate(sentences_vectors_sg):
    print(f"Vetor da Sentença {i+1}: {vector}")

# Word2Vec CBOW

### Vetores pré-treinados

Uso de vetores pré-treinados como ponto de partida para o treinamento do modelo CBOW

In [ ]:
cbow = "../src/GoogleNews-vectors-negative300.bin"

modelo_cbow = KeyedVectors.load_word2vec_format(cbow, binary=True)


### Testes

Testando a similaridade entre duas palavras usando o modelo pré-treinado para avaliar sua utilidade antes de proceder com o treinamento do nosso próprio modelo.

In [ ]:
wordvec_word1 = modelo_cbow['dog']

wordvec_word1

In [ ]:
wordvec_word2 = modelo_cbow['puppy']

wordvec_word2

In [ ]:
1 - cosine(wordvec_word1, wordvec_word2)

### Vetores da nossa base

In [ ]:
# Assuming df['comment'] contains tokenized sentences
sentences = df['words'].tolist()

# Train Word2Vec model using summed sentence vectors
model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the model in word2vec format
model.wv.save_word2vec_format('word2vec_model.bin', binary=True)


In [ ]:
palavra1 = 'like'
palavra2 = 'hate'

similaridade = modelo_cbow.similarity(palavra1, palavra2)
print(f'A similaridade entre "{palavra1}" e "{palavra2}" é: {similaridade}')

### Função para somar os vetores de palavras da base

Como nossa base é pautada em frases e não em palavras soltas, para que o modelo word2ved faça mais sentido, nós devemos fazer uma função para somar as palavras presentes em determinada frase e assim conseguir um vetor resultade da frase.

In [ ]:
# Carregar o modelo Word2Vec previamente treinado
model = KeyedVectors.load_word2vec_format('word2vec_model.bin', binary=True, unicode_errors='ignore')

# Supondo que df['comment'] contém sentenças tokenizadas
sentences = df['words'].tolist()

# Função para somar os vetores de palavras de uma frase
def sum_word_vectors(sentence):
    # Inicializar um vetor de zeros com a mesma dimensão dos vetores de palavras no modelo
    sentence_vector = np.zeros(model.vector_size)
    
    # Contagem do número de palavras na frase que estão no vocabulário do modelo
    words_count = 0
    
    # Para cada palavra na frase, se ela estiver no vocabulário do modelo, somar seu vetor ao vetor da frase
    for word in sentence:
        if word in model:
            sentence_vector += model[word]
            words_count += 1
    
    # Normalizar o vetor da frase dividindo pela contagem de palavras
    if words_count != 0:
        sentence_vector /= words_count
    
    return sentence_vector

# Calcular os vetores das frases
sentences_vectors = [sum_word_vectors(sentence) for sentence in sentences]

# Printar os vetores das frases
for i, (sentence, vector) in enumerate(zip(sentences, sentences_vectors)):
    print(f"Frase {i+1}:")
    print(f"  Tokens: {' '.join(sentence)}")
    print(f"  Vetor: {vector}")
    print()


### Resultado e Análise dos Testes

**Desempenho dos Modelos**

- Word2Vec com Dados Pré-processados: Os modelos treinados com dados pré-processados (tokenizados e limpos) apresentaram melhor desempenho em comparação com os dados não processados.

- Vetores Pré-treinados vs. Dados da Base: Modelos que utilizaram vetores pré-treinados como ponto de partida superaram aqueles treinados apenas com nossa base de dados, devido à limitação da quantidade de dados disponíveis.

- CBOW vs. Skip-gram: Ambos os modelos apresentaram bons resultados. No entanto, o Skip-gram mostrou uma similaridade maior entre "like" e "hate", o que é justificado pelo mesmo ter sido treinado com nossa base de dados, que é limitada, enquanto o CBOW utiliza os vetores pré-treinados da Google.


#### Análise de Similaridade: like e hate

A seguir, foi feita uma análise e alguns testes para entender a alta similaridade entre as palavras "hate" e "like", o que parece inconsistente, mais detalhes acerca de possíveis razões para a alta similaridade estão descritos na seção 3.6.7 da documentação. 

**Passos para uma melhor análise**

1. Explorar os contextos das palavras

In [ ]:
like_contexts = df[df['comment'].str.contains('like', na=False)]['comment']
hate_contexts = df[df['comment'].str.contains('hate', na=False)]['comment']

print("Contextos de 'like':")
for context in like_contexts:
    print(context)

In [ ]:
print("\nContextos de 'hate':")
for context in hate_contexts:
    print(context)

2. Visualização dos Vetores

Utilizar técnicas de visualização como PCA (Análise de Componentes Principais) para projetar os vetores em um espaço de menor dimensão e visualizar a relação entre diferentes palavras.

In [ ]:
# Palavras para análise
words = ['like', 'hate', 'love', 'do not like']

# Filtrando palavras que estão no vocabulário do modelo
available_words = [word for word in words if word in model_sg]
word_vectors = [model_sg[word] for word in available_words]

# Verificar se todas as palavras foram encontradas no vocabulário
missing_words = set(words) - set(available_words)
if missing_words:
    print(f"As seguintes palavras não foram encontradas no vocabulário do modelo: {missing_words}")

# Realizar PCA para reduzir a dimensionalidade dos vetores
pca = PCA(n_components=2)
result = pca.fit_transform(word_vectors)

# Plotar os vetores
plt.scatter(result[:, 0], result[:, 1])

for i, word in enumerate(available_words):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))

plt.show()


Interpretação do Gráfico

1. Proximidade no Espaço Vetorial:

No gráfico, as palavras "like" e "hate" estão relativamente distantes uma da outra, enquanto "love" está mais separada de ambas.
A distância entre "like" e "hate" no espaço vetorial indica que, apesar da alta similaridade calculada inicialmente, no espaço de vetores de menor dimensão, elas não são tão próximas. Isso sugere que a similaridade pode ser influenciada por outros fatores além da mera proximidade no espaço vetorial.

2. Análise de Contexto:

A proximidade das palavras no gráfico pode refletir a maneira como elas são usadas no corpus. "Love" e "hate" são palavras com significados opostos e, idealmente, deveriam estar mais distantes uma da outra. No entanto, a similaridade calculada pode indicar que elas compartilham contextos emocionais intensos, mesmo sendo diferentes.

A visualização indica que "like" e "love" estão mais próximos, o que é esperado, pois ambas expressam sentimentos positivos.

3. Avaliar a Influência de Parâmetros do Modelo

Variar parâmetros como vector_size, window, min_count, e epochs para ver como eles afetam a similaridade entre "like" e "hate".

In [ ]:
model_sg_tuned = Word2Vec(sentences, vector_size=200, window=10, min_count=2, sg=1, workers=4, epochs=5)
similaridade_tuned = model_sg_tuned.wv.similarity('like', 'hate')
print(f'A similaridade entre "like" e "hate" após ajustar parâmetros é: {similaridade_tuned}')

3. Comparar com Modelos Diferentes

In [ ]:
model_fasttext = FastText(sentences, vector_size=100, window=5, min_count=1, sg=1, workers=4, epochs=10)
similaridade_fasttext = model_fasttext.wv.similarity('like', 'hate')
print(f'A similaridade entre "like" e "hate" usando FastText é: {similaridade_fasttext}')

**Conclusão**

A conclusão geral sobre os resultados está detalhada na seção 3.6.7 da documentação. Os pontos abordados são: similaridade calculada, visualização dos vetores, influência do contexto semântico, importância do pré-processamento e parâmetros.

# Modelo


## 2. Dividir os Dados em Conjuntos de Treino e Teste

Nessa fase, separamos os dados coletados em dois grupos: um para treinar nosso modelo e outro para testá-lo depois. Isso ajuda a garantir que nosso modelo aprenda com um conjunto de dados e seja capaz de fazer boas previsões sobre dados novos e desconhecidos.

In [ ]:
# Convertendo para numpy array para facilitar a manipulação
X = np.array(sentences_vectors)
y = df['sentiment']

# Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## 3. Treinar os Modelos


Nesta etapa, usamos dois tipos diferentes de modelos para entender melhor os sentimentos expressos nas avaliações do Uber:

**Naive Bayes**

Este modelo é bom para lidar com palavras e textos. Ele assume que cada palavra contribui de forma independente para o sentimento da avaliação, o que nos ajuda a calcular a probabilidade de um comentário ser positivo ou negativo.

In [ ]:
# Inicializar e treinar o modelo Naive Bayes
bayes_model = GaussianNB()
bayes_model.fit(X_train, y_train)

**Svm**

Este modelo tenta encontrar a melhor linha (ou fronteira) que separa os comentários positivos dos negativos. É como desenhar uma linha reta no meio de pontos em um gráfico para distinguir entre duas categorias.

In [ ]:
param_grid = {
    'C': [0.1, 1, 10, 100], 
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 
    'degree': [2, 3, 4], 
    'gamma': ['scale', 'auto'] 
}

svm_model = SVC()
svm_model.fit(X_train, y_train)

# Use GridSearchCV para encontrar os melhores hiperparâmetros
grid_search = GridSearchCV(estimator=svm_model, param_grid=param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)


In [ ]:
print(grid_search.best_params_)
best_svm_model = grid_search.best_estimator_
best_svm_model.fit(X_train, y_train)

## 4. Avaliar o Modelo Naive Bayes


Após treinar nossos modelos, o próximo passo é testar como eles performam com dados que eles nunca viram antes. Para isso, seguimos estes passos:

1. **Prever os Sentimentos no Conjunto de Teste**: Usamos o modelo Naive Bayes, que foi treinado anteriormente, para prever os sentimentos nas avaliações do conjunto de teste. Essas previsões são armazenadas em `y_pred`.

2. **Calcular a Precisão**: A precisão é uma medida que nos diz qual porcentagem das previsões estava correta. Calculamos isso usando a função `accuracy_score`, comparando as previsões `y_pred` com as verdadeiras respostas `y_test`.

3. **Gerar e Imprimir o Relatório de Classificação**: O relatório de classificação fornece mais detalhes sobre a performance do modelo, como precisão, recall e a pontuação F1 para cada classe (positivo, negativo). Usamos a função `classification_report` para obter e imprimir este relatório.

Essas métricas nos ajudam a entender quão bem o modelo está trabalhando e em quais áreas ele pode ser melhorado.

In [ ]:
# Prever os sentimentos no conjunto de teste
y_pred = bayes_model.predict(X_test)

# Calcular a precisão
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Gerar e imprimir o relatório de classificação
print(classification_report(y_test, y_pred))


## 5. Avaliar Modelo SVM

Depois de treinar o modelo SVM, precisamos verificar como ele se comporta com dados que não foram usados no treinamento. Veja os passos a seguir:

1. **Prever os Sentimentos no Conjunto de Teste**: Utilizamos o modelo SVM para fazer previsões sobre as avaliações no conjunto de teste. Guardamos essas previsões na variável `y_pred_svm`.

2. **Calcular a Precisão**: Para entender quão precisas são essas previsões, calculamos a precisão usando a função `accuracy_score`. Essa função compara as previsões `y_pred_svm` com as respostas reais `y_test` para calcular a porcentagem de acertos.

3. **Gerar e Imprimir o Relatório de Classificação**: Para uma análise mais detalhada, geramos um relatório de classificação com a função `classification_report`. Esse relatório mostra a precisão, o recall e a pontuação F1 para cada classe de sentimento (positivo, negativo). Isso nos ajuda a entender melhor as forças e fraquezas do modelo em classificar corretamente os sentimentos.

Esse processo ajuda a avaliar a eficácia do modelo SVM em identificar corretamente os sentimentos nas avaliações, fornecendo insights valiosos para possíveis ajustes no modelo.

In [ ]:
# Prever os sentimentos no conjunto de teste
y_pred = best_svm_model.predict(X_test)

# Calcular a precisão
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Gerar e imprimir o relatório de classificação
print(classification_report(y_test, y_pred))

## Conclusão
O modelo SVM Linear, usando a técnica de bigrams no processo de BoW, foi o mais eficaz para a análise de sentimentos das avaliações do Uber. Essa abordagem alcançou uma precisão geral de 76.43% com resultados sólidos em todas as categorias de sentimentos. Continuaremos refinando o modelo para melhorar sua precisão e capacidade de generalização.